Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import preprocessing
import h5py

Load segmentation map from dataset (HOWS_CL_25)
The dataset should be at 2D_Shape_Completion/HOWS_CL_25

In [4]:
file = h5py.File("HOWS_CL_25/Images/apple/0/0a2af0e9d0d746169e712d2fddbd61ba/0.hdf5", "r")
segmap = file["segmap"]
np_segmap = np.array(segmap)
binary_image = preprocessing.segmap_to_binary(np_segmap)
surface = preprocessing.get_surface_pixels(binary_image)
sampled_surface = preprocessing.sample_pixels(surface, 50)
# visualize image, surface and sampled surface as images side by side
fig, ax = plt.subplots(1, 3)
ax[0].imshow(binary_image, cmap="gray")
ax[0].set_title("Binary Image")
ax[1].imshow(surface, cmap="gray")
ax[1].set_title("Surface")
ax[2].imshow(sampled_surface, cmap="gray")
ax[2].set_title("Sampled Surface")
#save binary_image to folder segmaps/apple/0/0a2af0e9d0d746169e712d2fddbd61ba/0.png
image_path = "segmaps/apple/0/0a2af0e9d0d746169e712d2fddbd61ba"
os.makedirs(image_path, exist_ok=True)
plt.imsave(image_path + "/0.png", binary_image, cmap="gray")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'HOWS_CL_25/Images/apple/0/0a2af0e9d0d746169e712d2fddbd61ba/0.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Would be good to extract the segmentation maps from all images as this is basically the only data we need, no RGB images or depth maps...
Next step: Train a modified U-Net model to predict a shape from a sparse set of sampled surface points.
Code for creating the dataset:

In [ ]:
# Function to process and save segmentation maps and surface samples
def process_and_save(file_path, save_path, prefix):
    file = h5py.File(file_path, "r")
    segmap = file["segmap"]
    np_segmap = np.array(segmap)
    binary_image = preprocessing.segmap_to_binary(np_segmap)
    surface = preprocessing.get_surface_pixels(binary_image)
    
    # Save binary image
    os.makedirs(save_path, exist_ok=True)
    plt.imsave(os.path.join(save_path, f"{prefix}_binary_image.png"), binary_image, cmap="gray")
    
    # Save surface image
    plt.imsave(os.path.join(save_path, f"{prefix}_surface.png"), surface, cmap="gray")
    
    # Save sampled surfaces
    for num_samples in [10, 15, 20, 30, 50]:
        sampled_surface = preprocessing.sample_pixels(surface, num_samples)
        plt.imsave(os.path.join(save_path, f"{prefix}_sampled_surface_{num_samples}.png"), sampled_surface, cmap="gray")

# Walk through all .hdf5 files in the dataset directory
dataset_dir = "HOWS_CL_25/Images"
prefix = 0
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        if file.endswith(".hdf5"):
            file_path = os.path.join(root, file)
            relative_path = os.path.relpath(root, dataset_dir)
            save_path = os.path.join("data", relative_path, str(prefix))
            process_and_save(file_path, save_path, prefix)
            prefix += 1

In [5]:
import os
import shutil

# Percorso della cartella principale
root_folder = "/percorso/della/cartella"

# Percorso della cartella di output
output_folder = "/percorso/cartella/output"
os.makedirs(output_folder, exist_ok=True)

# Funzione per spostare file e rimuovere cartelle vuote
def move_files_and_cleanup(folder):
    for root, dirs, files in os.walk(folder, topdown=False):  # Usa topdown=False per rimuovere cartelle vuote dal basso verso l'alto
        for file in files:
            file_path = os.path.join(root, file)
            # Sposta il file nella cartella di output
            shutil.move(file_path, os.path.join(output_folder, file))
        
        # Rimuove le cartelle vuote
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            if not os.listdir(dir_path):  # Controlla se la cartella è vuota
                os.rmdir(dir_path)

# Esegui la funzione
move_files_and_cleanup(root_folder)


In [14]:
import torch
from my_unet_model import UNet # Importa la definizione del modello

# Ricrea il modello
model = UNet(1,4,1)
model.load_state_dict(torch.load("model_weights.pth"))
model.eval()  # Imposta il modello in modalità di valutazione


UNet(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): 

In [20]:
import torch

# Specify the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the model and map it to the GPU
model = torch.load("model_full.pth", map_location=device)

# Set the model to evaluation mode
model.eval()

print("Model loaded onto", device)



Model loaded onto cpu


In [31]:
from PIL import Image
import numpy as np
import torch
import preprocessing
import matplotlib.pyplot as plt

# Load the test image
image_path = r"C:\Users\iacop\Desktop\Programmazione\Github\tum-adlr-11\data\55_surface.png"
image = Image.open(image_path).convert('L')  # Convert to grayscale
image = np.array(image)

# Sample pixels if needed (ensure preprocessing is defined correctly)
sampled_surface = preprocessing.sample_pixels(image, 50)

# Convert to torch tensor and add batch and channel dimensions
sampled_surface =  torch.tensor(sampled_surface, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, height, width]
norm_image = (sampled_surface-7.720400996428604e-07)/ 5.4754757846909005e-05
# Assuming the model is defined and loaded
model.eval()  # Set the model to evaluation mode (disables dropout, batch norm updates)
with torch.no_grad():  # Disable gradient calculation for evaluation
    y = model(sampled_surface)  # Get the model's predictions



imagex = torch.tensor(image).unsqueeze(0).unsqueeze(0) 

criterion = torch.nn.CrossEntropyLoss()
print("Error is", criterion(y, imagex))

# Post-process model output: remove batch and channel dimensions, convert to numpy
y = y.squeeze().cpu().numpy()  # Assuming the model output is on the GPU, use .cpu()
binary_output = (y > 0.5)

# Plot the images
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Display the predicted image
ax[0].imshow(binary_output, cmap="gray")
ax[0].set_title("Prediction")
# Assuming sampled_surface has shape [1, 1, height, width]
sampled_surface = sampled_surface.squeeze(0).squeeze(0)  # Removes the first two dimensions

# Display the ground truth image
ax[1].imshow(sampled_surface, cmap="gray")  # Ground truth
ax[1].set_title("Ground Truth")

# Display the original input image
ax[2].imshow(image, cmap='gray')  # Original input
ax[2].set_title("Input Image")

# Show the plot
plt.show()





torch.Size([1, 1, 224, 224]) torch.Size([1, 1, 224, 224])


RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 4

In [1]:
import os

# Specifica la cartella
folder_path = r"C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data"

# Scorri i file nella cartella
for filename in os.listdir(folder_path):
    # Controlla se il nome del file corrisponde al formato desiderato
    if filename.endswith("_binary_image.png"):  # Aggiungi l'estensione se necessario (es. .png, .jpg)
        # Costruisci il percorso completo del file
        file_path = os.path.join(folder_path, filename)
        # Elimina il file
        os.remove(file_path)
        print(f"Deleted: {file_path}")


Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\10047_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\10079_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\1007_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\100983_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\101019_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\101027_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\101224_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\101421_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\101622_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/tum-adlr-11/data\102056_binary_image.png
Deleted: C:/Users/iacop/Desktop/Programmazione/Github/